In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x / 255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def load_dataset(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, "images/*")))
    masks = sorted(glob(os.path.join(dataset_path, "masks/*")))

    train_x, test_x = train_test_split(images, test_size=0.2, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=0.2, random_state=42)

    return (train_x, train_y), (test_x, test_y)

def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path = image_path.decode()
        mask_path = mask_path.decode()

        x = read_image(image_path)
        y = read_mask(mask_path)

        return x, y

    image, mask = tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    image.set_shape([256, 256, 3])
    mask.set_shape([256, 256, 1])

    return image, mask

def tf_dataset(images, masks, batch=4):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

In [ ]:
!pip install kaggle

In [ ]:
import os

# Set Kaggle API credentials via environment variables
os.environ['KAGGLE_USERNAME'] = "jeanpierregergie"
os.environ['KAGGLE_KEY'] = "7187ca4ca21ea85ecfa3aa08a80326f9"

In [ ]:
!kaggle datasets download -d nikhilroxtomar/person-segmentation


 98% 453M/461M [00:08<00:00, 36.6MB/s]
100% 461M/461M [00:08<00:00, 58.6MB/s]


In [ ]:
!unzip person-segmentation.zip

Streaming output truncated to the last 5000 lines.
  inflating: people_segmentation/masks/pexels-photo-108155.png  
  inflating: people_segmentation/masks/pexels-photo-108157.png  
  inflating: people_segmentation/masks/pexels-photo-108261.png  
  inflating: people_segmentation/masks/pexels-photo-108292.png  
  inflating: people_segmentation/masks/pexels-photo-109002.png  
  inflating: people_segmentation/masks/pexels-photo-109394.png  
  inflating: people_segmentation/masks/pexels-photo-109547_rIJj622db8.png  
  inflating: people_segmentation/masks/pexels-photo-109669.png  
  inflating: people_segmentation/masks/pexels-photo-10979.png  
  inflating: people_segmentation/masks/pexels-photo-109851.png  
  inflating: people_segmentation/masks/pexels-photo-10988.png  
  inflating: people_segmentation/masks/pexels-photo-109917.png  
  inflating: people_segmentation/masks/pexels-photo-109920.png  
  inflating: people_segmentation/masks/pexels-photo-110080.png  
  inflating: people_segmentati

In [ ]:
dataset_path = "/content/people_segmentation"


In [ ]:
(train_x, train_y), (test_x, test_y) = load_dataset(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=4)

Train: 4542 - 4542
Test: 1136 - 1136


In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model


input_shape = (512, 512, 3)
model = build_unet(input_shape)
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0][0

In [ ]:
### import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

input_shape = (512, 512, 3)
model = build_unet(input_shape)

if __name__ == "__main__":
    """ Hyperparamaters """
    dataset_path = "/content/people_segmentation"
    input_shape = (256, 256, 3)
    batch_size = 30
    epochs = 30
    lr = 1e-4
    model_path = "unet.h5"
    csv_path = "data.csv"

    """ Load the dataset """
    (train_x, train_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    test_dataset = tf_dataset(test_x, test_y, batch=batch_size)

    """ Model """
    model = build_unet(input_shape)
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr),
        metrics=[
            tf.keras.metrics.MeanIoU(num_classes=2),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.Precision()
        ]
    )


    callbacks = [
        ModelCheckpoint(model_path, monitor="val_loss", verbose=1),
        ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor="val_loss", patience=10)
    ]

    train_steps = len(train_x)//batch_size
    if len(train_x) % batch_size != 0:
        train_steps += 1

    test_steps = len(test_x)//batch_size
    if len(test_x) % batch_size != 0:
        test_steps += 1

    model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=test_steps,
        callbacks=callbacks
    )


Train: 4542 - 4542
Test: 1136 - 1136
Epoch 1/30
152/152 [==============================] - ETA: 0s - loss: 0.4003 - mean_io_u: 0.3722 - recall: 0.5888 - precision: 0.6696
Epoch 1: saving model to unet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


152/152 [==============================] - 363s 2s/step - loss: 0.4003 - mean_io_u: 0.3722 - recall: 0.5888 - precision: 0.6696 - val_loss: 0.7559 - val_mean_io_u: 0.3721 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/30
152/152 [==============================] - ETA: 0s - loss: 0.2968 - mean_io_u: 0.3722 - recall: 0.7194 - precision: 0.7699
Epoch 2: saving model to unet.h5
152/152 [==============================] - 273s 2s/step - loss: 0.2968 - mean_io_u: 0.3722 - recall: 0.7194 - precision: 0.7699 - val_loss: 0.7753 - val_mean_io_u: 0.3721 - val_recall: 0.0210 - val_precision: 0.8810 - lr: 1.0000e-04
Epoch 3/30
152/152 [==============================] - ETA: 0s - loss: 0.2657 - mean_io_u: 0.3722 - recall: 0.7582 - precision: 0.7952
Epoch 3: saving model to unet.h5
152/152 [==============================] - 271s 2s/step - loss: 0.2657 - mean_io_u: 0.3722 - recall: 0.7582 - precision: 0.7952 - val_loss: 0.3398 - val_mean_io_u: 0.3721 - val_recall: 0.7308 

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm


""" Load the test images """
test_images = glob("/kaggle/input/sport/*")
""" Load the model """
model = tf.keras.models.load_model("unet.h5")
for path in tqdm(test_images, total=len(test_images)):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    original_image = x
    h, w, _ = x.shape
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    pred_mask = model.predict(x)[0]
    pred_mask = np.concatenate(
        [
            pred_mask,
            pred_mask,
            pred_mask
        ], axis=2)
    pred_mask = (pred_mask > 0.5) * 255
    pred_mask = pred_mask.astype(np.float32)
    pred_mask = cv2.resize(pred_mask, (w, h))
    original_image = original_image.astype(np.float32)
    alpha = 0.6
    cv2.addWeighted(pred_mask, alpha, original_image, 1-alpha, 0, original_image)
    show(original_image/255)
    name = path.split("/")[-1]
    cv2.imwrite(f"save_images/{name}", original_image)

0it [00:00, ?it/s]
